<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js001_ArMD2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [4]:
#@title js001_ArMD2D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: http://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.08.16 created,  last updated on 2024.04.06
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''



<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js001_ArMD2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js001_ArMD2D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.04.25 created, last updated on 2018.11.10
//    ver 0.0.1  2019.01.06 v1, last updated on 2021.04.20
//    ver 0.0.2  2021.10.27 v2, last updated on 2021.10.27
//    ver 0.0.3  2023.02.11 v3, last updated on 2023.08.11
//
// --------------------  molecular dynamics 2D
//
//    time evolution: velocity Verlet Algorithm
//      (1) vi = vi + (Fi/mi)*(0.5dt)
//      (2) ri = ri + vi*dt
//      (3) calculation Fi <- {r1,r2,...,rn} Fi=sum(Fij,j=1 to n,j!=i), Fij=F(ri-rj)
//      (4) vi = vi + (Fi/mi)*(0.5dt)
//      goto (1)
//
//    potential: Lennard-Jones V(r) = 4.0*epsilon*((sigma/r)^12-(sigma/r)^6)
//    force: F(r) = -dV(r)/dr = 24.0*epsilon*r6*(2.0*r6-1.0)/r, r6=(sigma/r)^6
//
// --------------------
*/

const ArMD2D = (function(){ // ====================  ArMD2D Module  ====================

	const g_AMU = 1.66053904e-27;			// (kg) atomic mass unit
	const g_kB = 1.380649e-23;				// (J/K) Boltzmann's constant
	const g_nMax = 500;						// array max

	let g_nParticles = 300;					// number of particles
	let g_sysTime = 0.0;					// (s) system time
	let g_timeStep = 5.0e-15;				// (s) time step dt
	let g_xMax = 8.0E-9;					// (m) x-box size
	let g_yMax = 8.0E-9;					// (m) y-box size
	let g_kineticEnergy = 0.0;				// (J) total kinetic energy
	let g_potentialEnergy = 0.0;			// (J) total potential energy

	const g_xx = dim1( g_nMax );			// (m) x-component of i-th particle position
	const g_yy = dim1( g_nMax );			// (m) y-component of i-th particle position
	const g_vx = dim1( g_nMax );			// (m/s) x-component of i-th particle velocity
	const g_vy = dim1( g_nMax );			// (m/s) y-component of i-th particle velocity
	const g_ffx = dim1( g_nMax );			// (N) x-component of total force applied i-th particle
	const g_ffy = dim1( g_nMax );			// (N) y-component of total force applied i-th particle

	function dim1( n ) {
		return new Float64Array( n );
	}

	// ---  material data

	const g_mass = 39.95*g_AMU;			// (kg) mass of Ar
	const g_sigma = 3.418e-10;			// (m) Lennard-Jones potential sigma for Ar
	const g_epsilon = 1.711e-21;		// (J) Lennard-Jones potential epsilon for Ar


	// --------------------  set initial condition  --------------------

	function setInitialCondition( nn, BoxSizeInNM, contTemp ) {

		g_sysTime = 0.0;
		g_nParticles = nn;
		g_xMax = BoxSizeInNM*1.0e-9;
		g_yMax = BoxSizeInNM*1.0e-9;
		g_nParticles = setParticles( nn );
		ajustVelocity(contTemp);
	}

	function setParticles( nn ) {
		const sigma = g_sigma;
		let nParticles = nn;

		for ( let j=0; j<nn; j++ ) {
			let loopCount = 0;
			let i;
			do {
				g_xx[j] = (g_xMax-1.2*sigma)*Math.random() + 0.6*sigma;
				g_yy[j] = (g_yMax-1.2*sigma)*Math.random() + 0.6*sigma;
				for ( i=0; i<j; i++ ) {
					if ((g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j]) < 1.4*sigma*sigma) break;
				}
				loopCount = loopCount + 1
				if (loopCount>1000) break;
			} while (i<j);
			if (loopCount>1000) {
				nParticles = j - 1;
				break;
			}
		}
		for ( let i=0; i<nParticles; i++ ) {
			g_vx[i] = 200.0*normalRandom3();
			g_vy[i] = 200.0*normalRandom3();
			g_ffx[i] = 0.0;
			g_ffy[i] = 0.0;
		}
		return nParticles;
	}

	// normal distributed random number: -3.0 <= normalRandom3() < 3.0
	function normalRandom3() {
		return (Math.random()+Math.random()+Math.random()+Math.random()+Math.random()+Math.random()-3.0);
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( nCalc ) {
		for (let i=0; i<nCalc; i++) {
			g_sysTime += g_timeStep;
			moveParticles(g_timeStep);
		}
	}

	function moveParticles( dt ) {
		const nn=g_nParticles;
		let a = 0.5*dt/g_mass;
		for (let i=0; i<nn; i++) {
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_xx[i] += g_vx[i]*dt;
			g_yy[i] += g_vy[i]*dt;
		}
		calcForce();
		for (let i=0; i<nn; i++) {
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
		}
		g_kineticEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_kineticEnergy += 0.5*g_mass*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
	}

	function calcForce() {
		const nn = g_nParticles, sgm = g_sigma, eps = g_epsilon;
		const s05 = 0.5*g_sigma;

		g_potentialEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_ffx[i]=0; g_ffy[i]=0;
		}
		for (let i=0; i<nn; i++) {
			for (let j=i+1; j<nn; j++) {
				const xij=g_xx[i]-g_xx[j];
				const yij=g_yy[i]-g_yy[j];
				const r = Math.sqrt(xij*xij+yij*yij);
				// calc. L-J force and potential
				const ri = sgm/r;
				const r6 = ri*ri*ri*ri*ri*ri;
				// V(r) = 4.0*epsilon*((sigma/r)^12-(sigma/r)^6)
				g_potentialEnergy += 4.0*eps*r6*(r6-1.0);
				// F(r) = 24.0*epsilon*r6*(2.0*r6-1.0)/r, r6=(sigma/r)^6
				const f = 24.0*eps*r6*(2.0*r6-1.0)/r;

				const fxij = f*xij/r;
				const fyij = f*yij/r;
				g_ffx[i] += fxij;
				g_ffy[i] += fyij;
				g_ffx[j] -= fxij;
				g_ffy[j] -= fyij;
			}
		}
		for (let i=0; i<nn; i++) {
			g_ffx[i] += boundaryForce(g_xx[i]+s05) + boundaryForce(g_xx[i]-g_xMax-s05);
			g_ffy[i] += boundaryForce(g_yy[i]+s05) + boundaryForce(g_yy[i]-g_yMax-s05);
		}
	}

	function boundaryForce(r) { // boundary:L-J type; epsilon = 0.5*epsilonOfAr, sigma = sigmaOfAr
		const ri = (g_sigma/r);
		const r6 = ri*ri*ri*ri*ri*ri;
		g_potentialEnergy += 4.0*0.5*g_epsilon*r6*(r6-1.0);
		return (24.0*0.5*g_epsilon*r6*(2.0*r6-1.0)/r);
	}


	// --------------------  utility  --------------------

	function systemTemperature() {
		const nn=g_nParticles;
		let ek= 0.0; // kinetic energy (J)
		for (let i=0; i<nn; i++) {
			ek = ek + 0.5*g_mass*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
		return ek/(nn*g_kB); // for 2D
	}

	function ajustVelocity(temp) {
		const nn = g_nParticles;
		const a = Math.sqrt(temp/systemTemperature());
		for (let i=0; i<nn; i++) {
			g_vx[i] = a*g_vx[i];
			g_vy[i] = a*g_vy[i];
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( nn, BoxSizeInNM, contTemp )
		evolve:			timeEvolution,			// timeEvolution( nCalc )

		setTemp:		ajustVelocity,			// ajustVelocity( contTemp )

		getSysParam:	function() { return [ g_nParticles, g_timeStep, g_xMax, g_yMax ]; },
		getNow:			function() { return [ g_sysTime, systemTemperature(), g_kineticEnergy, g_potentialEnergy ]; },
		getArData:		function() { return [ g_mass, g_sigma, g_epsilon, 'Ar' ]; },

		getPosition:	function(i) { return [ g_xx[i], g_yy[i] ]; },
		getVelocity:	function(i) { return [ g_vx[i], g_vy[i] ]; },
		getForce:		function(i) { return [ g_ffx[i], g_ffy[i] ]; },
	};

})(); // ====================  ArMD2D Module end  ====================


const js001 = (function(){ // ====================  js Module  ====================

	const theModule = ArMD2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;
	let ctx;

	let v_nn = 100;
	let v_BoxSizeInNM = 8.0;
	let v_tempMode = 0;			// 0: adiabatic, 1: temp.control
	let v_contTemp = 300.0;

	let p_nParticles, p_timeStep, p_xMax, p_yMax; // = theModule.getSysParam();
	let sysTime, temperature, kineticEnergy, potentialEnergy;
	let nowData = [];
  let kindList = [];
  let xxList = [];
  let yyList = [];
  let vxList = [];
  let vyList = [];

	let dispMode = 0;			// no use
	let resetFlag = true;
	let pauseFlag = false;

	let breakFlag = false;
	let perticleFlag = true;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_nn, v_BoxSizeInNM, v_contTemp ); // ( nn, BoxSizeInNM, contTemp )
			[ p_nParticles, p_timeStep, p_xMax, p_yMax ] = theModule.getSysParam();
			perticleFlag = true;
		}

		if ( !pauseFlag ) {
			const nCalc = 40;
			theModule.evolve( nCalc );
		}

		draw( ctx, dispMode );

		if ( perticleFlag ) setParticlesData();

		requestAnimationFrame(animate);
	}

	function setParticlesData() {
		nowData = [ sysTime, temperature, kineticEnergy, potentialEnergy ];
    for (let i=0; i<p_nParticles; i++) {
			let x, y, vx, vy;
      kindList[i] = 2; // 2: Ar
			[ x, y ] = theModule.getPosition(i);
			xxList[i] = x;
			yyList[i] = y;
			[ vx, vy ] = theModule.getVelocity(i);
			vxList[i] = vx;
			vyList[i] = vy;
		}
	}


	// --------------------  draw  --------------------

	const molecColor = '#b4ff00';

	function draw( ctx, dispMode ) {
		const xp = 40, yp = 5, xBoxSize = 400, yBoxSize = 400, yTextPos = yCanvasSize-45;

		ctx.clearRect( 0, 0, xCanvasSize, yCanvasSize );

		// draw box
		ctx.strokeStyle = "#888800";
		ctx.strokeRect( xp, yp, xBoxSize, yBoxSize );

		// draw ball
		drawBalls( ctx, xp, yp, xBoxSize/p_xMax );

		// caption
		[ sysTime, temperature, kineticEnergy, potentialEnergy ] = theModule.getNow();
		const energy = kineticEnergy+potentialEnergy;

		ctx.fillStyle = "#888888";
		ctx.fillText( `time = ${(sysTime*1.0e12).toFixed(1)} (ps)`, xp, yTextPos );
		ctx.fillText( `Temp = ${temperature.toFixed(1)} (K)`, xp+200, yTextPos );
		ctx.fillText( `molec: Ar, N = ${p_nParticles}`, xp, yTextPos+20 );
		ctx.fillText( `energy = ${energy.toExponential(4)} (J)`, xp+200, yTextPos+20 );
		ctx.fillText( `Box = ${(p_xMax*1.0e9).toFixed(1)+"x"+(p_yMax*1.0e9).toFixed(1)} (nm)`, xp, yTextPos+40 );
		//document.getElementById("text_caption").innerHTML = "minimum molecular dynamics code"
	}

	function drawBalls( ctx, xp, yp, scale ) {
		const yMax = p_yMax, nn = p_nParticles, twoPi = 2.0*Math.PI;
		const r = 0.5*theModule.getArData()[1]; // r = 0.5*sigma of Ar

		ctx.fillStyle = molecColor;
		for (let i=0; i<nn; i++) {
			ctx.beginPath();
			let x, y;
			[ x, y ] = ArMD2D.getPosition(i);
			ctx.arc(x*scale+xp, (yMax-y)*scale+yp, r*scale, 0, twoPi, false);
			ctx.fill();
		}
	}


	// --------------------  control  --------------------

	function pause() {
		pauseFlag = ( pauseFlag==false );
	}

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTemperature(temp) {
    theModule.setTemp(temp);
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, temperature, kineticEnergy, potentialEnergy ];
  }

  function pygetParticlesList() {
    perticleFlag = false;
    return [ nowData, kindList, xxList, yyList, vxList, vyList ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		pause:			pause,			// pause();

    breakLoop: breakLoop, // breakLoop();
    pysetTemperature: pysetTemperature, // pysetTemperature( temp )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, temperature, kineticEnergy, potentialEnergy ]
		pygetParticlesList, pygetParticlesList, //() :return [ nowData, kindList, xxList, yyList, vxList, vyList ]
	};

})(); // ====================  js001 module end  ====================


const js = js001;
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js001] Ar in the Box (Molecular Dynamics 2D)</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="480"></canvas><br>

<button onClick="js.pause()">pause/go</button><br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# exec html-js code, and python control

import time

# exec html-js code
exec_html_js()
print("-- start --")

# python control
for i in range(10):
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print(f' i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  energy = {energy:9.6e} (J)')
  time.sleep(3)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / gas cool down

import time
import matplotlib.pyplot as plt

# exec html-js code
exec_html_js()
print("-- start --")

# python control

timeList = []
tempList = []
energyList = []

for i in range(20):
  # control system temperature
  contTemp = 300.0 - 15.0*i
  eval_js('js.pysetTemperature({})'.format(contTemp))
  time.sleep(3)
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  timeList.append(sysTime*1e12)
  tempList.append(sysTemp)
  energyList.append(energy*1e20)
  if i%2==0:
    print(f' i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  energy = {energy:9.6e} (J)')
  time.sleep(1)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

plt.plot( timeList, tempList, 'xb-' )
plt.xlabel('time (ps)')
plt.ylabel('temp (K)')
plt.show()

plt.plot( timeList, energyList, 'og-' )
plt.xlabel('time (ps)')
plt.ylabel('energy ( x10^-20 J )')
plt.show()

In [ ]:
# calab AI wrote:
# prompt: plot tempList, energyList in one figure with dual y-axis

import matplotlib.pyplot as plt

# Create a figure and axes
fig, ax1 = plt.subplots()

# Plot temperature on the primary y-axis
ax1.plot(timeList, tempList, 'xb-', label='Temperature')
ax1.set_xlabel('Time (ps)')
ax1.set_ylabel('Temperature (K)')

# Create a twin axes for energy
ax2 = ax1.twinx()

# Plot energy on the secondary y-axis
ax2.plot(timeList, energyList, 'og-', label='Energy')
ax2.set_ylabel('Energy (x10^-20 J)')

# Add a legend
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

# Display the plot
plt.show()


In [ ]:
# get particles data

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print(f' i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  energy = {energy:9.6e} (J)')
  time.sleep(1)

# get particle data
print("-- get particles data --")
[ nowData, kindList, xxList, yyList, vxList, vyList ] = eval_js('js.pygetParticlesList()')
[ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = nowData
energy = kineticEnergy + potentialEnergy
print(f'got time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  energy = {energy:9.6e} (J)')
print("list length = ", len(vxList))
print(f'len(xxList) ={len(xxList):>4d}, len(yyList) ={len(yyList):>4d}, len(vxList) ={len(vxList):>4d},  len(vyList) ={len(vyList):>4d} \n')

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# save particles data

import numpy as np

# convert [ kindList, xxList, yyList, vxList, vyList ] to np_data
np_data = np.array([ kindList, xxList, yyList, vxList, vyList ])
print("shape of  np_data :", np_data.shape )

# save np_data
print("-- save as 'js001_data.npy'" )
np.save( 'js001_data.npy', np_data )

# The saved file can be viewed by clicking on the folder icon on the left edge of the notebook.
# You must download this file to save it permanently.
# Select Download from the '...'.T menu of the file to download it.

In [ ]:
# load particles data

import numpy as np

# load particles data
print("-- load data --")
loaded_data = np.load('js001_data.npy')

# set numpy array
Kind = loaded_data[0].astype(int)  # kind of every atom
X = loaded_data[1] * 1e9           # (nm) x-position of every atom
Y = loaded_data[2] * 1e9           # (nm) y-position of every atom
U = loaded_data[3]                 # (m/s) x-velocity of every atom
V = loaded_data[4]                 # (m/s) y-velocity of every atom
Vabs = np.sqrt(U**2 + V**2)        # (m/s) speed of every atom

In [ ]:
# histogram of Vabs

import matplotlib.pyplot as plt

plt.hist(Vabs)

In [ ]:
# plot particle position

import numpy as np
import matplotlib.pyplot as plt

# scatter plot
fig = plt.figure(figsize=(8, 8))
plt.scatter(X,Y, c=Vabs, cmap='jet', s=3.42*50) # Ar size = 3.42e-10
plt.xlabel('x ( nm )')
plt.ylabel('y ( nm )')
plt.title('Ar in the box')
plt.xlim(0,8)
plt.ylim(0,8)
plt.show()

In [ ]:
# velocity space

import numpy as np
import matplotlib.pyplot as plt

# scatter plot
fig = plt.figure(figsize=(10, 8))
plt.scatter(U, V, c=Vabs, cmap='jet')
plt.xlabel('vx ( m/s )')
plt.ylabel('vy ( m/s )')
plt.title('velocity space')
plt.colorbar()
plt.show()